In [12]:
import base64
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

In [3]:
def load_image(image_path: str) -> dict:
    """Load image from file and encode it as base64."""
    image_path = image_path
  
    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
        
    image_base64 = encode_image(image_path)
    return {"image": image_base64}

In [4]:
image_dict = load_image("data/chips.jpg")

In [5]:
image_dict

{'image': '...'}

In [7]:
llm = ChatOpenAI(temperature=0.5, model="gpt-4-turbo")

In [23]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ImageInformation(BaseModel):
    """Information about an image."""
    image_description: str = Field(description="a short description of the image")
    is_complaint_genuine: bool = Field(description="Is the complaint genuine")
    count: int = Field(description="Count of images")

parser = JsonOutputParser(pydantic_object=ImageInformation)

In [24]:
vision_prompt = """
   A custome has complained saying ```I asked for fries with a little garlic sauce but it has arrived with way too much sauce```. Provide me the following information:
    - A description of the image
    - Is the complaint genuine
    - Count of images
   """

In [27]:
msg = llm.invoke(
             [HumanMessage(
             content=[
             {"type": "text", "text": vision_prompt},
             {"type": "text", "text": parser.get_format_instructions()},
             {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_dict['image']}"}},
             ])]
             )

BadRequestError: Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}

In [ ]:
msg

AIMessage(content='```json\n{\n  "image_description": "The image shows a cardboard container filled with fries that are excessively covered in white garlic sauce, almost completely obscuring the fries beneath.",\n  "is_complaint_genuine": true,\n  "count": 1\n}\n```', response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 2503, 'total_tokens': 2558}, 'model_name': 'gpt-4-turbo', 'system_fingerprint': 'fp_0737e0dfd9', 'finish_reason': 'stop', 'logprobs': None}, id='run-7db63d22-99ca-4f9c-84f5-917a25a91a1d-0')